In [6]:
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
from bson import ObjectId
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sshtunnel import SSHTunnelForwarder
from dbutils.pooled_db import PooledDB

ssh_host = 'api.trustyapp.io'
ssh_username = 'ubuntu'
ssh_pem_file = './ng-prod.pem'
database_host = '127.0.0.1'  # SSH 터널을 통해 localhost로 접근
database_name = 'trusty'
database_username = 'server'
database_password = 'ygFAhsyTMDzB21VdLTnoq7djxwjDNzuc9yPGukm73otXER8ihD'

server = SSHTunnelForwarder(
    (ssh_host, 22),  # SSH 서버 주소와 포트
    ssh_username=ssh_username,
    ssh_pkey=ssh_pem_file,  # Private key 파일 경로
    remote_bind_address=(database_host, 5432)  # 원격 MySQL 서버 주소와 포트
)

server.start()

conn = psycopg2.connect(
    host='127.0.0.1',  # localhost를 통해 SSH 터널로 접속
    user=database_username,
    password=database_password,
    database=database_name,
    port=server.local_bind_port  # sshtunnel이 자동으로 할당한 포트
)
cursor = conn.cursor()

In [ ]:
conn.rollback()

qry = """
select *
from bank_transaction bt 
left join bank_account ba on ba.id = bt."bankAccountId" 
where ba."userId" = 418
order by bt."transactionAt" asc,  bt.id asc
        """
cursor.execute(qry)
result = cursor.fetchall()

# DataFrame 생성
df = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])
df.to_csv('./418.csv', index=False, encoding='utf-8-sig')

# 결과 확인
print(df.head())

                         createdAt                        updatedAt     id  \
0 2024-11-22 22:17:48.846660+00:00 2024-11-22 22:17:48.846660+00:00  62305   
1 2024-11-22 22:17:48.846660+00:00 2024-11-22 22:17:48.846660+00:00  62304   
2 2024-11-22 22:17:48.846660+00:00 2024-11-22 22:17:48.846660+00:00  62302   
3 2024-11-22 22:17:48.846660+00:00 2024-11-22 22:17:48.846660+00:00  62303   
4 2024-11-22 22:17:48.846660+00:00 2024-11-22 22:17:48.846660+00:00  62301   

  serviceProvider             transactionId    type  amount  \
0            mono  6741030728ffa62e52afd7de  credit       9   
1            mono  6741030728ffa62e52afd7dd   debit     261   
2            mono  6741030728ffa62e52afd7db   debit  260000   
3            mono  6741030728ffa62e52afd7dc  credit  260000   
4            mono  6741030728ffa62e52afd7da   debit  250000   

              transactionAt       description  bankAccountId  ...  \
0 2023-12-01 01:35:32+00:00  OWealth Interest             98  ...   
1 2023-12-01 0

In [10]:
conn.rollback()

qry = """
select ch."bureau", ch."reportedAt", ch."identifiedInstitution", ch."accountNumber", ch."loanAmount", ch."repaymentAmount", ch."openedAt", ch."dueAt", ch."lastRepaymentAt", ch."status"
from credit_history ch
where ch."userId" IN ('83', '131', '658', '655', '450', '783')
        """
cursor.execute(qry)
result = cursor.fetchall()

# DataFrame 생성
df = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])
df.to_csv('./꾼.csv', index=False, encoding='utf-8-sig')


# 결과 확인
print(df.head())

          bureau  reportedAt             identifiedInstitution  accountNumber  \
0  first-central  2024-11-18  money in minutes finance company  2022-45151485   
1  first-central  2024-11-18  money in minutes finance company  2022-46591196   
2  first-central  2024-11-18  money in minutes finance company  2022-52305292   
3  first-central  2022-03-07      mycredit investments limited         100991   
4  first-central  2022-08-08      mycredit investments limited        5308901   

  loanAmount repaymentAmount    openedAt       dueAt lastRepaymentAt  \
0       5000            7000  2022-08-28  2022-11-09      2022-09-06   
1      20000           26000  2022-09-10        None            None   
2      10000           13500  2022-09-06        None      2022-09-10   
3     200000           58330  2021-12-05  2022-06-03      2022-02-04   
4     200000           58330  2021-12-05  2022-07-05      2022-05-22   

       status  
0  performing  
1        lost  
2  performing  
3  performing  


In [11]:
conn.rollback()

qry = """
select ch."bureau", ch."reportedAt", ch."identifiedInstitution", ch."accountNumber", ch."loanAmount", ch."repaymentAmount", ch."openedAt", ch."dueAt", ch."lastRepaymentAt", ch."status"
from credit_history ch
where ch."userId" IN ('679', '675', '673', '670', '87', '103', '372', '389', '435', '530', '598', '712', '793', '113')
        """
cursor.execute(qry)
result = cursor.fetchall()

# DataFrame 생성
df = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])
df.to_csv('./꾼아님.csv', index=False, encoding='utf-8-sig')


# 결과 확인
print(df.head())

          bureau  reportedAt              identifiedInstitution  \
0  first-central  2022-12-08              access bank plc lagos   
1  first-central  2019-12-12              access bank plc lagos   
2  first-central  2024-10-18              credit direct limited   
3  first-central  2024-10-14   source microfinance bank limited   
4  first-central  2024-11-14  bravewood finance company limited   

      accountNumber loanAmount repaymentAmount    openedAt       dueAt  \
0  099LARL213100EBJ      11017            None  2021-11-06  2022-11-01   
1  099PDLP19252D2FX       1000               0  2019-09-09  2019-10-09   
2       PSB14478792     157000           19127  2024-03-27  2025-03-28   
3           1002055     200000           28382  2024-06-14  2025-07-10   
4             90481     189000           19120  2024-03-21  2025-06-28   

  lastRepaymentAt      status  
0            None  performing  
1      2019-10-14  performing  
2      2024-08-01  performing  
3      2024-06-14  perfo